In [1]:
import pandas as pd

# Read and Prep Data

In [1]:
# !ls
!dir

 Volume in drive C is Windows
 Volume Serial Number is 4333-1BF2

 Directory of C:\Users\Gaston\Desktop\Bayesian_OPt\BayesianOptimization

03/20/2020  04:46 PM    <DIR>          .
03/20/2020  04:46 PM    <DIR>          ..
03/16/2020  03:20 PM    <DIR>          .ipynb_checkpoints
03/16/2020  02:54 PM           640,216 Business_case_dataset.csv
03/20/2020  04:43 PM            21,744 RandomForestBayes.ipynb
03/20/2020  04:46 PM               453 README.md
03/20/2020  04:44 PM            37,984 TensorflowBayesian.ipynb
               4 File(s)        700,397 bytes
               3 Dir(s)  958,555,918,336 bytes free


In [6]:
actual_labels=[
    "ID",
    "Book_Length(mins)_overall",
    "Book_Length(mins)_avg",
    "Price_overall",
    "Price_avg",
    "Gave_Review",
    "Review 10/10",
    "Minutes_listened",
    "%_completion",
    "Support_Requests",
    "Last_visited_minus_purchase_date",
    "targets"
  ];
raw_data = pd.read_csv("Business_case_dataset.csv",names= actual_labels)

True

In [15]:
raw_data.count()

ID                                  14083
Book_Length(mins)_overall           14083
Book_Length(mins)_avg               14083
Price_overall                       14083
Price_avg                           14083
Gave_Review                         14083
Review 10/10                        14083
Minutes_listened                    14083
%_completion                        14083
Support_Requests                    14083
Last_visited_minus_purchase_date    14083
targets                             14083
dtype: int64

In [5]:
raw_data.head()

ID  Book_Length(mins)_overall  Book_Length(mins)_avg  Price_overall  \
0  873                     2160.0                   2160          10.13   
1  611                     1404.0                   2808           6.66   
2  705                      324.0                    324          10.13   
3  391                     1620.0                   1620          15.31   
4  819                      432.0                   1296           7.11   

   Price_avg  Gave_Review  Review 10/10  Minutes_listened  %_completion  \
0      10.13            0          8.91               0.0           0.0   
1      13.33            1          6.50               0.0           0.0   
2      10.13            1          9.00               0.0           0.0   
3      15.31            0          9.00               0.0           0.0   
4      21.33            1          9.00               0.0           0.0   

   Support_Requests  Last_visited_minus_purchase_date  targets  
0                 0                                 0        1  
1                 0                               182        1  
2                 1                               334        1  
3                 0                               183        1  
4                 0                                 0        1

In [22]:
df = raw_data.dropna()
df.count()

ID                                  14083
Book_Length(mins)_overall           14083
Book_Length(mins)_avg               14083
Price_overall                       14083
Price_avg                           14083
Gave_Review                         14083
Review 10/10                        14083
Minutes_listened                    14083
%_completion                        14083
Support_Requests                    14083
Last_visited_minus_purchase_date    14083
targets                             14083
dtype: int64

In [23]:
y= df.iloc[:,-1:].copy()
y.head()

targets
0        1
1        1
2        1
3        1
4        1

In [24]:
X = df.iloc[:,1:-1].copy()
X.head()

Book_Length(mins)_overall  Book_Length(mins)_avg  Price_overall  Price_avg  \
0                     1404.0                   2808           6.66      13.33   
1                      324.0                    324          10.13      10.13   
2                     1620.0                   1620          15.31      15.31   
3                      432.0                   1296           7.11      21.33   
4                     2160.0                   2160          10.13      10.13   

   Gave_Review  Review 10/10  Minutes_listened  %_completion  \
0            1           6.5               0.0           0.0   
1            1           9.0               0.0           0.0   
2            0           9.0               0.0           0.0   
3            1           9.0               0.0           0.0   
4            1           9.0               0.0           0.0   

   Support_Requests  Last_visited_minus_purchase_date  
0                 0                               182  
1                 1                               334  
2                 0                               183  
3                 0                                 0  
4                 0                                 5

In [25]:
X_act= pd.get_dummies(X)
X_act.count()

Book_Length(mins)_overall           14083
Book_Length(mins)_avg               14083
Price_overall                       14083
Price_avg                           14083
Gave_Review                         14083
Review 10/10                        14083
Minutes_listened                    14083
%_completion                        14083
Support_Requests                    14083
Last_visited_minus_purchase_date    14083
dtype: int64

In [26]:
import numpy as np
y_act= np.array(y)
y_act= y_act.reshape((-1,))
y_act.shape

(14083,)

# Split data into Train, Val, Test sets

In [27]:
from sklearn.model_selection import train_test_split
#Perform the split
X_train, X_val, y_train, y_val = train_test_split(X_act, y_act, test_size=0.2, random_state=0)

In [28]:
from sklearn.ensemble import RandomForestClassifier
from skopt import BayesSearchCV

# Perform Optimization

In [40]:
opt = BayesSearchCV(
    RandomForestClassifier(n_jobs=-1),
    {
        'n_estimators': (1, 200), # int range
        'max_depth' : [None,1,2,3,4,5,6,7], # categorical
        'min_samples_split' : (2,5),
        'min_samples_leaf' : (1,10),
        

    },
    n_iter=32,
    cv=5
)

In [41]:
import warnings
warnings.filterwarnings('ignore', message='The objective has been evaluated at this point before.')
opt.fit(X_train, y_train)

BayesSearchCV(cv=5, error_score='raise',
              estimator=RandomForestClassifier(bootstrap=True,
                                               class_weight=None,
                                               criterion='gini', max_depth=None,
                                               max_features='auto',
                                               max_leaf_nodes=None,
                                               min_impurity_decrease=0.0,
                                               min_impurity_split=None,
                                               min_samples_leaf=1,
                                               min_samples_split=2,
                                               min_weight_fraction_leaf=0.0,
                                               n_estimators='warn', n_jobs=-1,
                                               oob_score=False,
                                               random_state=None, verbose=0,
                                   

In [42]:
print("val. score: %s" % opt.best_score_)
print("test score: %s" % opt.score(X_val, y_val))

val. score: 0.9100834368897568
test score: 0.9222577209797657
